# <center> <img src="/figs/LogoUFSCar.jpg" alt="Logo UFScar" width="110" align="left"/>  <br/> <center>Universidade Federal de São Carlos (UFSCar)<br/><font size="4"> Departamento de Computação, campus Sorocaba</center></font>
</p>

<font size="4"><center><b>Disciplina: Novas Tecnologias em Banco de Dados</b></center></font>
  
<font size="3"><center>Profa. Dra. SAHUDY MONTENEGRO GONZÁLEZ</center></font>

## <center>Projeto Final</center>

*INTEGRANTES*

**Integrante 01**: Laura Rieko Marçal Imai

**RA**: 

**Integrante 02**: Pedro Enrico Barchi Nogueira

**RA**: 813099


# Extração e Integração de Dados de Publicações Químicas via Web Scraping

## Sumário
1. [Introdução](#introducao)
2. [Inicialização e Preparação do Ambiente](#inicializacao-e-preparacao-do-ambiente)
3. [Extração dos Dados Básicos (articles.csv)](#extracao-dos-dados-basicos)
4. [Adição da métrica Total Access](#adicao-total-access)
5. [Análise Exploratória](#analise-exploratoria)
6. [Tratamento dos Dados para o Data Warehouse](#tratamento-dados)

## <a id="introducao"></a>Introdução

Neste projeto, desenvolvemos um pipeline de web scraping para extrair e integrar dados de publicações científicas de duas importantes revistas químicas: **Química Nova (QN)** e **Journal of the Brazilian Chemical Society (JBCS)**. O objetivo é coletar informações básicas dos artigos (como título, autores, data de publicação, etc.) e, posteriormente, complementar estes dados com a métrica "Total Access" diretamente dos sites das revistas no portal SBQ. 


## <a id="inicializacao-e-preparacao-do-ambiente"></a>Inicialização e Preparação do Ambiente

Nesta seção, preparamos o ambiente de trabalho importando as bibliotecas necessárias e configurando os scripts que compõem o pipeline do projeto. O projeto foi desenvolvido em **Python 3.12.4** e utiliza módulos como `requests` e `BeautifulSoup` para o scraping, além de ferramentas para manipulação e visualização dos dados. A estrutura do projeto está organizada em scripts separados, permitindo que as funções principais (para extração dos dados básicos e para o complemento do 'Total Access') sejam chamadas de forma modular a partir deste notebook.

### Instalando Pacotes Necessários


In [ ]:
# Pacotes para manipulação de dados e web scraping
%pip install requests beautifulsoup4 pandas numpy matplotlib seaborn unidecode rapidfuzz

### Importações e Configuração Inicial

In [1]:
import csv
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import logging
from datetime import datetime
import ast

# Configuração do logger para acompanhamento do fluxo
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
)
logger = logging.getLogger("ScraperPublicaçõesQuímicas")

# Importação dos scripts do projeto

from scrapers.scraper_basico import run_scraper

from scrapers.scraper_total_access import run_total_access

from tratamento.tratamento_dados import tratar_dados

## <a id="extracao-dos-dados-basicos"></a>Extração dos Dados Básicos (articles.csv)

Nesta etapa, utilizamos o módulo `scraper_basico.py` para extrair as informações fundamentais de publicações das revistas Química Nova (QN) e Journal of the Brazilian Chemical Society (JBCS) a partir da SciELO.  
O resultado deste processo é armazenado em um arquivo CSV intermediário chamado **articles.csv**.  
Esta etapa realiza a extração dos dados básicos, como título, autores, data de publicação, entre outros.

In [ ]:
# Executa o scraper para extrair os artigos
logger.info("Iniciando a extração dos dados básicos (articles.csv)...")
artigos = run_scraper()  # run_scraper() retorna uma lista de dicionários com os dados dos artigos

logger.info(f"Total de artigos extraídos: {len(artigos)}")

# Visualiza os 3 primeiros artigos para verificação
for idx, art in enumerate(artigos[:3], start=1):
    logger.info(f"Artigo {idx}: {art}")

# Salva os dados extraídos no arquivo 'articles.csv'
output_csv_path = "articles.csv"
fieldnames = [
    "journal",
    "year",
    "volume",
    "edition_number",
    "publication_date",
    "publication_type",
    "title",
    "authors",
    "keywords",
    "institutions"
]

logger.info(f"Salvando os dados extraídos em {output_csv_path}...")
with open(output_csv_path, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for art in artigos:
        # Caso os campos como authors, keywords, institutions estejam como lista, converta-os em string
        art["authors"] = "; ".join(art["authors"]) if isinstance(art["authors"], list) else art["authors"]
        art["keywords"] = "; ".join(art["keywords"]) if isinstance(art["keywords"], list) else art["keywords"]
        art["institutions"] = "; ".join(art["institutions"]) if isinstance(art["institutions"], list) else art["institutions"]
        if isinstance(art.get("publication_date"), (datetime,)):
            art["publication_date"] = art["publication_date"].strftime("%Y-%m-%d")
        writer.writerow(art)
logger.info(f"Extração concluída! {output_csv_path} foi criado com sucesso.")

## <a id="adicao-total-access"></a>Adição da métrica Total Access

Agora que já extraímos as informações básicas das publicações e armazenamos no arquivo `articles.csv`, iremos complementar esses dados com a métrica **Total Access**.

O **Total Access** representa o número de acessos de cada artigo e será obtido diretamente dos sites das revistas **Química Nova (QN)** e **Journal of the Brazilian Chemical Society (JBCS)**. Para isso, utilizamos o módulo `scraper_total_access.py`, que busca as edições correspondentes de cada periódico e extrai as informações de acesso para cada artigo.

Ao final dessa etapa, um novo arquivo CSV atualizado será gerado: **articles_final.csv**.


In [ ]:
# Define os caminhos dos arquivos de entrada e saída
input_csv = "articles.csv"
output_csv = "articles_final.csv"

print("Iniciando o processo de compleção do 'Total Access'...")
run_total_access(input_csv, output_csv)
print(f"Processo concluído! Arquivo gerado: {output_csv}")

# Exibir os primeiros registros do CSV final para verificação
import pandas as pd
df_final = pd.read_csv(output_csv)
df_final.head(10)

## <a id="analise-exploratoria"></a>Análise Exploratória dos Dados

Nesta etapa, vamos realizar uma análise exploratória básica do arquivo **articles_final.csv** que contém os dados integrados das publicações com a métrica **TotalAccess**. Com isso, temos o objetivo de verificar a integridade dos dados e identificar registros com valores nulos em **TotalAccess**, exibir estatísticas descritivas da métrica de acesso e visualizar a distribuição dos acessos com gráficos.

Em seguida, removeremos os registros com valor nulo em **TotalAccess** para o tratamento posterior dos dados.


In [ ]:
# Carregar o dataset final
df = pd.read_csv("articles_final.csv")
print("Número de registros originais:", len(df))

# Converter a coluna 'TotalAccess' para numérico; registros não convertíveis serão definidos como NaN
df['TotalAccess'] = pd.to_numeric(df['TotalAccess'], errors='coerce')

# Verificar a quantidade de registros com TotalAccess nulo
missing_total = df['TotalAccess'].isnull().sum()
print("Registros com TotalAccess nulo:", missing_total)

# Exibir uma amostra dos registros com TotalAccess nulo (se houver)
if missing_total > 0:
    print("Exemplo de registros com TotalAccess nulo:")
    display(df[df['TotalAccess'].isnull()].head())

# Remover registros onde TotalAccess é nulo
df_clean = df.dropna(subset=['TotalAccess'])
print("Número de registros após remover nulos:", len(df_clean))

# Exibir estatísticas descritivas da coluna TotalAccess
print("\nEstatísticas descritivas de TotalAccess:")
print(df_clean['TotalAccess'].describe())

# Histograma da distribuição de TotalAccess
plt.figure(figsize=(10, 6))
sns.histplot(df_clean['TotalAccess'], bins=30, kde=True, color='skyblue')
plt.title("Distribuição do TotalAccess")
plt.xlabel("TotalAccess")
plt.ylabel("Frequência")
plt.show()

## <a id="tratamento-dados"></a> Tratamento dos Dados para o Data Warehouse

Nesta etapa, aplicamos operações de limpeza e transformação para padronizar e preparar os dados das publicações para inserí-los no Data Warehouse. As operações incluem:

- Padronização dos nomes das instituições (remoção de acentos, conversão para minúsculas e eliminação de duplicatas);
- Normalização das palavras-chave, convertendo-as para minúsculas e removendo acentos;
- Mapeia as palavras-chave para subáreas da química e adiciona a coluna "subareas";
- Separação e padronização dos nomes dos autores;
- Conversão das datas para um formato consistente (YYYY-MM-DD) e extração dos componentes ano, mês e dia;
- Criação de um identificador único para cada edição, concatenando volume e número;
- Remoção de registros duplicados (mesmo título) entre as revistas.

Após o processamento, os dados serão salvos em um novo arquivo CSV que poderá ser carregado no DW.


In [3]:

# Define os caminhos para o CSV de entrada e saída
input_csv = "articles_final.csv"
output_csv = "articles_dw.csv"

# Executa o processo de tratamento
tratar_dados(input_csv, output_csv)

Dados tratados e salvos em articles_dw.csv
